In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os, sys
from itertools import product
USER = os.getenv("USER")
sys.path.append(f'/home/{USER}/repos/agriculture/1_code/5_crop_shift')
from agval.utils.ag_weights import snag_weights
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context('poster')

/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:38235")
client

<Client: 'tcp://127.0.0.1:38235' processes=10 threads=80, memory=251.59 GiB>

In [3]:
slr=5
year=2099
output='/shares/gcp/integration/CE_library_epa_vsl_bc39/synthetic_CAMEL/'

In [4]:
gdppc = xr.open_zarr(
    "/shares/gcp/integration/float32/dscim_input_data/econvars/zarrs/integration-econ-bc39.zarr"
).sel(year=year).gdppc

AMEL = xr.open_zarr(
    f"/shares/gcp/integration/CE_library_epa_vsl_bc39/AMEL/adding_up_ce_cc_eta2.0.zarr"
).sel(year=year)

AMEL['gcm'] = AMEL.gcm.str.upper()

weights =[]
for rcp in ['rcp45', 'rcp85']:
    weights.append(snag_weights(rcp).to_xarray().expand_dims({'rcp' : [rcp]}))
    
weights = xr.concat(weights, 'rcp').weight.fillna(0)

AMEL = AMEL.weighted(weights).mean('gcm')

coastal = xr.open_zarr(
    f"/shares/gcp/integration/CE_library_epa_vsl_bc39/coastal/adding_up_ce_cc_eta2.0.zarr"
).sel(slr=slr, year=year)

coastal = coastal.reindex({"region": AMEL.region.values}, fill_value=np.nan)
coastal = xr.where(np.isnan(coastal), gdppc, coastal)

# xr.open_zarr("/shares/gcp/integration/float32/dscim_input_data/climate/gmsl_distribution/sliiders-slr-v1.0.zarr")

In [5]:
CAMEL = (AMEL.ce_cc + coastal.ce_cc - gdppc).rename('ce_cc').to_dataset().expand_dims({'year' : [year]}).chunk({'region' : len(gdppc.region.values)})

for var in CAMEL.variables:
    CAMEL[var].encoding.clear()

In [6]:
CAMEL.to_zarr(
    f"{output}/CAMEL_weighted_gcm_slr{slr}_{year}_adding_up_ce_cc_eta2.0.zarr", 
    consolidated=True,
    mode='w'
)